# Cycle GAN

In [1]:
from keras.layers import *
from keras.models import Sequential, Model
from keras.optimizers import Adam
import matplotlib.pyplot as plt
import numpy as np

/home/sitibanc/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Define Class

In [ ]:
class CycleGAN():
    def __init__(self, img_shape, cycle_weight=10., lr=2e-4):
        '''
        Build a simple CycleGan object
        Args:
            img_shape: tuple, input image shape (channels included)
            cycle_weight: float, weight of cycle consistency loss, default=10.
            lr: float, learning rate, default=2e-4
        Returns:
            CycleGAN object
        '''
        # Get variables
        self.img_shape = img_shape
        self.optimizer = Adam(lr=lr)
        # Get networks
        self.d_x = self.build_discriminator()
        self.d_y = self.build_discriminator()
        self.g_x2y = self.build_generator()
        self.g_y2x = self.build_generator()
        # Compile discriminators
        self.d_x.compile(loss="binary_crossentropy",
                         optimizer=self.optimizer, metrics=["accuracy"])
        self.d_y.compile(loss="binary_crossentropy",
                         optimizer=self.optimizer, metrics=["accuracy"])
        # Combined discrminators and generators:
        # get images(,update discriminators) then fix discriminators and get predictions
        # Define input shape
        real_x = Input(shape=self.img_shape, name="x_input")
        real_y = Input(shape=self.img_shape, name="y_input")
        # Transfer to new domain
        x2y = self.g_x2y(real_x)
        y2x = self.g_y2x(real_y)
        # Transfer to original domain
        x2y2x = self.g_y2x(x2y)
        y2x2y = self.g_x2y(y2x)
        # Freeze discriminators
        self.d_x.trainable = False
        self.d_y.trainable = False
        # Get predictions from discriminators
        x2y_pred = d_y(x2y)
        y2x_pred = d_y(y2x)
        # Combine GANs
        self.cycleGAN = Model(inputs=[real_x, real_y],
                              outputs=[
                                  x2y_pred, y2x_pred,    # adversarial loss
                                  x2y2x, y2x2y])    # cycle consistency loss
        # Compile entire model
        self.cycleGAN.compile(loss=["mse", "mse", "mae", "mae"])

    def build_discriminator(self):
        model = Sequential()

        # Convolution Layers
        model.add(Conv2D(64, kernel_size=3, strides=1,
                         input_shape=self.img_shape, padding="same", name="conv2d"))
        model.add(LeakyReLU(alpha=0.2, name=""))
        model.add(Dropout(0.2, name=""))

        model.add(Conv2D(128, kernel_size=3, strides=1, padding="same", name=""))
        model.add(LeakyReLU(alpha=0.2, name=""))
        model.add(Dropout(0.2, name=""))

        model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.2))

        model.add(Conv2D(512, kernel_size=3, strides=1, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.2))

        # Dense Layer
        model.add(Flatten())
        model.add(Dense(1))
        model.add(Activation("sigmoid"))

        model.summary()

    def build_generator(self):

    def train(self, x_train, y_train, epochs, batch_size, save_path, save_interval=50):
        '''
        Train model and save generated image and loss during training process
        Args:
            x_train: ndarray, training images of x domain
            y_train: ndarray, training images of y domain
            epochs: int, training epochs
            batch_size: int, training batch size
            save_path: str, saving path for generated images during training process
            save_interval: int, save generated images every save_interval, default=50
        Returns:
            None
        '''